In [93]:
import numpy as np
import pandas as pd
import string
import re

#sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

#scipy imports
from scipy.sparse import hstack

#Visualization imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import bokeh
#! pip install bokeh

# target classes
target_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [110]:
# read frames localy through csv
train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")
# train_data = train_df["comment_text"]
# train_labels = train_df[target_names]

# Random index generator for splitting training data
# Note: Each rerun of cell will create new splits.
randIndexCut = np.random.rand(len(train_df)) < 0.8

#S plit up data
test_data = test_df["comment_text"]
dev_data, dev_labels = train_df[~randIndexCut]["comment_text"], train_df[~randIndexCut][target_names]
train_data, train_labels = train_df[randIndexCut]["comment_text"], train_df[randIndexCut][target_names]

# starting from the /usr/share/dict/words set of half a million words with abbreviations
# and others, and adding to them any obvious missing values
filename = 'words.txt'
file = open(filename, 'rt')
all_words = file.read()
file.close
all_words = all_words.split()
all_words = [ word.lower() for word in all_words]
all_words = set(all_words)

from nltk.corpus import brown
good_words_list = brown.words()
good_word_set = set([word.lower() for word in good_words_list])
#punctuation = re.sub("[\'\-]",'',string.punctuation)
punctuation = "[\!\?\"\#\$\%\&\(\)\*\+\,\.\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~\']"

In [254]:
from autocorrect import spell

def find_spelling_errors(textdoc):
    words = textdoc.split()
    words = [ word.lower() for word in words]
    misspelled_words = []
    misspelled_words.append([re.sub(punctuation,'',word) for word in words if re.sub(punctuation,'',word) not in large_word_corpus])
    return misspelled_words

def trysplit(word):
    lentmp = len(word)
    return_words=[]
    for i in range(lentmp):
        if large_word_corpus.intersection(set([word[:lentmp-i].lower()])):
            return_words.append(word[:lentmp-i])
            if len(word[lentmp-i:]) > 0:
                return_words.extend(trysplit(word[lentmp-i:]))
            else:
                return [word[:lentmp-i]]
            break
    return return_words 

def fix_spelling_errors(textdoc):
    words = textdoc.split()
    return_list = []
    for word in words:
        if all_words.intersection(set([re.sub(punctuation,'',word.lower())])):
            return_list.append(word)
        else:
            # word is not found in the dictionary, try to correct the spelling
            if word == spell(word): # no changes made by the spell checker
                return_list.extend(trysplit(word))
            else:
                return_list.append(spell(word))
    return return_list
    #return [ spell(word.lower()) if re.sub(punctuation,'',word.lower()) not in all_words else word for word in words]
    #return [spell(word) for word in words]

In [255]:
# find all the spelling errors
spelling_errors=[]
for index in range(1):
    spelling_errors.append(find_spelling_errors(train_data[index]))

flat_list = []
for sublist in spelling_errors:
    for sublist2 in sublist:
        for item in sublist2:
            flat_list.append(item)
            
# y = np.bincount(np.unique(flat_list))
# ii = np.nonzero(y)[0]
# out = np.vstack((ii, y[ii])).T
import collections
collections.Counter(flat_list)
# print(np.unique(flat_list, return_counts=True))
# print(flat_list)
x=8
print(train_data[x])
print('Invalid spellings: %s' % (find_spelling_errors(train_data[x])))
print(fix_spelling_errors(train_data[x]))



Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169
Invalid spellings: [['articlewow', '93161107169']]
['Sorry', 'if', 'the', 'word', "'nonsense'", 'was', 'offensive', 'to', 'you.', 'Anyway,', "I'm", 'not', 'intending', 'to', 'write', 'anything', 'in', 'the', 'article', 'they', 'would', 'jump', 'on', 'me', 'for', 'vandalism),', "I'm", 'merely', 'requesting', 'that', 'it', 'be', 'more', 'encyclopedic', 'so', 'one', 'can', 'use', 'it', 'for', 'school', 'as', 'a', 'reference.', 'I', 'have', 'been', 'to', 'the', 'selective', 'breeding', 'page', 'but', "it's", 'almost', 'a', 's

In [123]:
def bernoulli_toxic_labels (label_vector):
    return [1 if (label_vector['toxic'][x] + label_vector['severe_toxic'][x] +
                label_vector['obscene'][x] + label_vector['threat'][x] +
                label_vector['insult'][x] + label_vector['identity_hate'][x]) > 0 else 0 
            for x in label_vector.index.values]

def binarize_toxic_labels (label_vector):
    return [(label_vector['toxic'][x]*32 + label_vector['severe_toxic'][x]*16 +
                label_vector['obscene'][x]*8 + label_vector['threat'][x]*4 +
                label_vector['insult'][x]*2 + label_vector['identity_hate'][x]) 
            for x in label_vector.index.values]

# Need to convert the training labels, either a 1 to indicate that some value was set
# or use the binary and create a 64 value training lables.  Eventually should split off
# one for each and have a separate classifier


binary_train_labels = binarize_toxic_labels(train_labels)
binary_dev_labels = binarize_toxic_labels(dev_labels)

count_vect = CountVectorizer(max_features=5000,stop_words='english')
tfidf_vect = TfidfVectorizer(max_features=5000,stop_words='english')
X_train_counts = count_vect.fit_transform(train_data)
X_dev_counts = count_vect.transform(dev_data)
X_train_tfidf = tfidf_vect.fit_transform(train_data)
X_dev_tfidf = tfidf_vect.transform(dev_data)

alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}

# -- Step 2 train a multinomial Bayes and find the optimal alpha --
multinomial_nb_class = MultinomialNB().fit(X_train_counts, binary_train_labels)
clf = GridSearchCV(multinomial_nb_class, param_grid = alphas)
clf.fit(X_train_counts, binary_train_labels)
print("The best value from the input set alpha is: ", clf.best_params_)
# I think get the f1 score for this alpha
predicted_labels = multinomial_nb_class.predict(X_dev_counts)
print('The F1 score for MultinomialNB with alpha %s is %f' 
       % (clf.best_params_, metrics.f1_score(binary_dev_labels,predicted_labels, average='micro')))


bernoulli_nb_class = BernoulliNB().fit(X_train_counts, binary_train_labels)
clf = GridSearchCV(bernoulli_nb_class, param_grid = alphas)
clf.fit(X_train_counts, binary_train_labels)
print("The best value from the input set alpha is: ", clf.best_params_)
# I think get the f1 score for this alpha
predicted_labels = bernoulli_nb_class.predict(X_dev_counts)
print('The F1 score for BernoulliNB with alpha %s is %f' 
       % (clf.best_params_, metrics.f1_score(binary_dev_labels,predicted_labels, average='micro')))


# gaussian_nb_class = GaussianNB().fit(X_train_counts, binary_train_labels)
# clf = GridSearchCV(gaussian_nb_class, param_grid = alphas)
# clf.fit(X_train_counts, binary_train_labels)
# print("The best value from the input set alpha is: ", clf.best_params_)
# # I think get the f1 score for this alpha
# predicted_labels = gaussian_nb_class.predict(X_dev_counts)
# print('The F1 score for GaussianNB with alpha %s is %f' 
#        % (clf.best_params_, metrics.f1_score(binary_dev_labels,predicted_labels, average='micro')))

The best value from the input set alpha is:  {'alpha': 10.0}
The F1 score for MultinomialNB with alpha {'alpha': 10.0} is 0.879169
The best value from the input set alpha is:  {'alpha': 10.0}
The F1 score for BernoulliNB with alpha {'alpha': 10.0} is 0.641099


In [122]:
for name in target_names:
    multinomial_nb_class = MultinomialNB().fit(X_train_counts, train_labels[name])
    predicted_labels_dev = multinomial_nb_class.predict(X_dev_counts)
    predicted_labels_train = multinomial_nb_class.predict(X_train_counts)
    print('The F1 score for train MultinomialNB %s is %f' 
       % (name, metrics.f1_score(train_labels[name],predicted_labels_train, average='micro')))
    fpr, tpr, thresholds = metrics.roc_curve(train_labels[name], predicted_labels_train)
    print('Train score for class {} is {}'.format(name, metrics.auc(fpr,tpr))) 
    print('The F1 score for dev MultinomialNB %s is %f' 
       % (name, metrics.f1_score(dev_labels[name],predicted_labels_dev, average='micro')))
    fpr, tpr, thresholds = metrics.roc_curve(dev_labels[name], predicted_labels_dev)
    print('Dev score for class {} is {}\n'.format(name, metrics.auc(fpr,tpr)))
   
    

The F1 score for train MultinomialNB toxic is 0.951274
Train score for class toxic is 0.842156401391639
The F1 score for dev MultinomialNB toxic is 0.946908
Dev score for class toxic is 0.812182953651046

The F1 score for train MultinomialNB severe_toxic is 0.987213
Train score for class severe_toxic is 0.7571986548632726
The F1 score for dev MultinomialNB severe_toxic is 0.987726
Dev score for class severe_toxic is 0.7044201879954483

The F1 score for train MultinomialNB obscene is 0.968099
Train score for class obscene is 0.8353591391246766
The F1 score for dev MultinomialNB obscene is 0.966802
Dev score for class obscene is 0.803674443269784

The F1 score for train MultinomialNB threat is 0.995704
Train score for class threat is 0.5282727952345427
The F1 score for dev MultinomialNB threat is 0.996471
Dev score for class threat is 0.5249549155285191

The F1 score for train MultinomialNB insult is 0.965449
Train score for class insult is 0.8118920284726971
The F1 score for dev Multino